In [2]:
# Imports and plotting setups
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import math

import sys, os
sys.path.insert(0, '../../')
sys.path.insert(0, '../')

import math_funcs

from cycler import cycler

default_cycler = (cycler(color=['#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377', 'black']) +
                cycler(linestyle=['-', '--', ':', '-.', '-', '--', ':']))

plt.rc('lines', linewidth=1)
plt.rc('axes', prop_cycle=default_cycler)

plt.rcParams["font.family"] = "Times"
plt.rcParams["font.size"] = 8

plt.rcParams['figure.figsize'] = (3.5, 2.5)
plt.rcParams['figure.dpi'] = 200

plt.rcParams['text.usetex'] = True

from scipy.constants import (
    epsilon_0,   # Permittivity of free space (vacuum)
    mu_0,        # Permeability of free space (vacuum)
    c,           # Speed of light in vacuum
    #e,           # Elementary charge
    #h,           # Planck constant
    #hbar,        # Reduced Planck constant (h-bar)
    k,           # Boltzmann constant
    #G,           # Newtonian constant of gravitation
    #m_e,         # Electron mass
    #m_p,         # Proton mass
    #m_n,         # Neutron mass
    #alpha,       # Fine-structure constant
    eV,          # Electron volt
)

# Example usage of some constants
print(f"Permittivity of free space: {epsilon_0} F/m")
print(f"Permeability of free space: {mu_0} N/A^2")
print(f"Speed of light in vacuum: {c} m/s")
# print(f"Elementary charge: {e} C")
# print(f"Planck constant: {h} J·s")
# print(f"Reduced Planck constant: {hbar} J·s")
print(f"Boltzmann constant: {k} J/K")
# print(f"Newtonian constant of gravitation: {G} m^3/kg/s^2")
# print(f"Electron mass: {m_e} kg")
# print(f"Proton mass: {m_p} kg")
# print(f"Neutron mass: {m_n} kg")
# print(f"Fine-structure constant: {alpha}")
print(f"Electron volt: {eV} J")

def print_engineering_notation(value):
    """
    Print a value in engineering notation (powers of ten in multiples of 3).

    Parameters:
    value (float): The value to be printed in engineering notation.
    """
    
    if value == 0:
        print("0")
        return

    exponent = int(math.floor(math.log10(abs(value)) / 3) * 3)
    mantissa = value / (10 ** exponent)
    
    print(f"{mantissa:.3f}e{exponent}")

# Example usage
print_engineering_notation(0)          # Output: 0
print_engineering_notation(1234.56)    # Output: 1.235e3
print_engineering_notation(0.00123456) # Output: 1.235e-3
print_engineering_notation(-987654)    # Output: -987.654e3
print_engineering_notation(1e9)        # Output: 1.000e9

Permittivity of free space: 8.8541878128e-12 F/m
Permeability of free space: 1.25663706212e-06 N/A^2
Speed of light in vacuum: 299792458.0 m/s
Boltzmann constant: 1.380649e-23 J/K
Electron volt: 1.602176634e-19 J
0
1.235e3
1.235e-3
-987.654e3
1.000e9


In [3]:
from transmission_reflection_coefficients import fresnel_coefficients
from pucks_to_penetrate_cryosphere import calc_path_loss

e:\Jake Sahli\Documents\Git\Europan_Cryosphere_Puck_Estimator\transmission_reflection_coefficients.py:19: SyntaxWarning: invalid escape sequence '\g'
  """
e:\Jake Sahli\Documents\Git\Europan_Cryosphere_Puck_Estimator\transmission_reflection_coefficients.py:103: SyntaxWarning: invalid escape sequence '\g'
  """
e:\Jake Sahli\Documents\Git\Europan_Cryosphere_Puck_Estimator\transmission_reflection_coefficients.py:178: SyntaxWarning: invalid escape sequence '\l'
  """Calculation for fresnel reflection coefficients
e:\Jake Sahli\Documents\Git\Europan_Cryosphere_Puck_Estimator\transmission_reflection_coefficients.py:208: SyntaxWarning: invalid escape sequence '\l'
  """
e:\Jake Sahli\Documents\Git\Europan_Cryosphere_Puck_Estimator\transmission_reflection_coefficients.py:253: SyntaxWarning: invalid escape sequence '\l'
  """
e:\Jake Sahli\Documents\Git\Europan_Cryosphere_Puck_Estimator\transmission_reflection_coefficients.py:303: SyntaxWarning: invalid escape sequence '\s'
  """
e:\Jake Sahl

In [6]:
def multipath_fading_in_planar_media(
    f,
    s_nominal = 103, #m
    d_ice_cave_placement_to_bed = 10, #m
    d_surface_placement_to_bed = 35, #m
    d_ice_cave_placement_to_surface = 25, #m
    d_surface_placement_to_surface = 1, #m
    x_path_length = 100 #m
):

    # Calculate the ray lengths
    def calc_multipath_length_and_angle(
        d_1, d_2, x_tot
        ):
        d_tot = d_1 + d_2
        theta = np.arctan(d_tot / x_tot)
        s_tot = d_tot / np.sin(theta)
        return s_tot, theta

    s_bed, theta_bed = calc_multipath_length_and_angle(
        d_ice_cave_placement_to_bed,
        d_surface_placement_to_bed,
        x_path_length
    )

    s_surf, theta_surf = calc_multipath_length_and_angle(
        d_ice_cave_placement_to_surface, 
        d_surface_placement_to_surface, 
        x_path_length   
    )

    # Calculate the magnitude of the reflections
    temperature_s = 273.13 - 4 # -4 C in K
    epsilon_s_prime = 3.1884 + 0.00091*(temperature_s - 273.13)

    epsilon_bed_prime = 6 # roughly for rocks
    epsilon_snow_prime = 1 # just gonna use air, and pretend like there is no snow for now

    R_v_bed, R_h_bed = fresnel_coefficients(np.sqrt(epsilon_s_prime), np.sqrt(epsilon_bed_prime), theta_bed)
    R_v_surf, R_h_surf = fresnel_coefficients(np.sqrt(epsilon_s_prime), np.sqrt(epsilon_snow_prime), theta_surf)

    # Calculate the attenuation along the path
    epsilon_s_primeprime = 10**(-3.0129 + 0.0123*(temperature_s - 273.13))

    micro_molarity_salt = 0

    sigma_s = 1e-6 * \
        (9 * np.e ** ((0.58*eV/k) * (1 / 258.15 - 1 / temperature_s)) \
        + 0.55 * micro_molarity_salt * np.e ** ((0.22*eV/k) * (1 / 258.15 - 1 / temperature_s)))

    alpha_s = calc_path_loss(f * np.pi * 2, epsilon_s_prime, epsilon_s_primeprime, sigma_s)


    # Calculate the path loss along the direct path
    def calc_all_path_loss(path_length, alpha_path, n=int(1e6)):
        linear_path_loss = 1
        for i in np.linspace(0, path_length, n):
            linear_path_loss *= np.e**(-2 * alpha_path * path_length / n)

        return linear_path_loss / (4 * np.pi * s_nominal)**2

    path_loss = calc_all_path_loss(s_nominal, alpha_s)
    # Calculate the path loss of the bed ref path
    path_loss_bed = calc_all_path_loss(s_bed, alpha_s)
    # Calculate the path loss of the surf ref path
    path_loss_surf = calc_all_path_loss(s_surf, alpha_s)

    # Calculate the phase of the nominal path
    def calc_phase_length(l, f):
        lambda_s = c / f / np.sqrt(epsilon_s_prime)
        return (2 * np.pi * l / lambda_s) % (2 * np.pi)

    phase_length = calc_phase_length(s_nominal, f)
    # Calculate the phase length of the bed ref path
    phase_length_bed = calc_phase_length(s_bed, f)

    # Calculate the phase length of the surf ref path
    phase_length_surf = calc_phase_length(s_surf, f)


    # Calculate the relevant magnitudes at the receiver
    nominal_mag = math_funcs.power_2_db(path_loss)

    multipath_mag = math_funcs.voltage_2_db(
        np.abs(np.sqrt(path_loss)*np.e**(1j*phase_length) \
            + np.sqrt(path_loss_bed)*np.e**(1j*phase_length_bed)\
                + np.sqrt(path_loss_surf)*np.e**(1j*phase_length_surf)))

    return nominal_mag, multipath_mag

f_uhf = 413e6 #Hz
f_hf = 5.373e6 #Hz

nominal_mag_uhf, multipath_mag_uhf = multipath_fading_in_planar_media(f_uhf)
nominal_mag_hf, multipath_mag_hf = multipath_fading_in_planar_media(f_hf)

f'at UHF {nominal_mag_uhf} dB -> {multipath_mag_uhf} dB and at HF {nominal_mag_hf} dB -> {multipath_mag_hf}',\
    f'UHF fading {nominal_mag_uhf - multipath_mag_uhf}, HF fading { nominal_mag_hf - multipath_mag_hf} '

('at UHF -66.58370296560663 dB -> -69.90556447259844 dB and at HF -64.72734867666458 dB -> -56.95719557748045',
 'UHF fading 3.3218615069918087, HF fading -7.770153099184128 ')

In [11]:
max_fading_uhf = -1 * np.inf
min_fading_uhf = np.inf
max_fading_hf = -1 * np.inf
min_fading_hf = np.inf
for d_ice_cave_placement_to_bed in np.linspace(5, 15, 25):
    for d_surface_placement_to_bed in np.linspace(30, 50, 25):
        nominal_mag_uhf, multipath_mag_uhf = multipath_fading_in_planar_media(
            f_uhf,
            d_ice_cave_placement_to_bed = d_ice_cave_placement_to_bed, #m
            d_surface_placement_to_bed = d_surface_placement_to_bed, #m
        )
        nominal_mag_hf, multipath_mag_hf = multipath_fading_in_planar_media(
            f_hf,
            d_ice_cave_placement_to_bed = d_ice_cave_placement_to_bed, #m
            d_surface_placement_to_bed = d_surface_placement_to_bed, #m
        )
        uhf_fading = nominal_mag_uhf - multipath_mag_uhf
        hf_fading = nominal_mag_hf - multipath_mag_hf

        if uhf_fading > max_fading_uhf:
            max_fading_uhf = uhf_fading
        
        if uhf_fading < min_fading_uhf:
            min_fading_uhf = uhf_fading

        if hf_fading > max_fading_hf:
            max_fading_hf = hf_fading
        
        if hf_fading < min_fading_hf:
            min_fading_hf = hf_fading

max_fading_uhf, max_fading_hf, min_fading_uhf, min_fading_hf

(np.float64(3.722509559646795),
 np.float64(-0.3494824835474333),
 np.float64(-8.242176933939362),
 np.float64(-9.208680872460384))